In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
from collections import Counter
import itertools

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

In [2]:
regions = {
    'albany': 'capital district',
    'allegany': 'western new york',
    'bronx': 'new york city',
    'broome': 'southern tier',
    'cattaraugus': 'western new york',
    'cayuga': 'central new york',
    'chautauqua': 'western new york',
    'chemung': 'southern tier',
    'chenango': 'southern tier',
    'clinton': 'north country',
    'columbia': 'capital district',
    'cortland': 'central new york',
    'delaware': 'southern tier',
    'dutchess': 'hudson valley',
    'erie': 'western new york',
    'essex': 'north country',
    'franklin': 'north country',
    'fulton': 'mohawk valley',
    'genesee': 'finger lakes',
    'greene': 'capital district',
    'hamilton': 'north country',
    'herkimer': 'mohawk valley',
    'jefferson': 'north country',
    'kings': 'new york city',
    'lewis': 'north country',
    'livingston': 'finger lakes',
    'madison': 'central new york',
    'monroe': 'finger lakes',
    'montgomery': 'mohawk valley',
    'nassau': 'long island',
    'new york': 'new york city',
    'niagara': 'western new york',
    'oneida': 'mohawk valley',
    'onondaga': 'central new york',
    'ontario': 'finger lakes',
    'orange': 'hudson valley',
    'orleans': 'finger lakes',
    'oswego': 'central new york',
    'otsego': 'mohawk valley',
    'putnam': 'hudson valley',
    'queens': 'new york city',
    'rensselaer': 'capital district',
    'richmond': 'new york city',
    'rockland': 'hudson valley',
    'saratoga': 'capital district',
    'schenectady': 'capital district',
    'schoharie': 'mohawk valley',
    'schuyler': 'southern tier',
    'seneca': 'finger lakes',
    'st lawrence': 'north country',
    'steuben': 'southern tier',
    'suffolk': 'long island',
    'sullivan': 'hudson valley',
    'tioga': 'southern tier',
    'tompkins': 'southern tier',
    'ulster': 'hudson valley',
    'warren': 'capital district',
    'washington': 'capital district',
    'wayne': 'finger lakes',
    'westchester': 'hudson valley',
    'wyoming': 'finger lakes',
    'yates': 'finger lakes'
}

geo_json_ids = {
    'albany': 'us-ny-001',
    'allegany': 'us-ny-003',
    'bronx': 'us-ny-005',
    'broome': 'us-ny-007',
    'cattaraugus': 'us-ny-009',
    'cayuga': 'us-ny-011',
    'chautauqua': 'us-ny-013',
    'chemung': 'us-ny-015',
    'chenango': 'us-ny-017',
    'clinton': 'us-ny-019',
    'columbia': 'us-ny-021',
    'cortland': 'us-ny-023',
    'delaware': 'us-ny-025',
    'dutchess': 'us-ny-027',
    'erie': 'us-ny-029',
    'essex': 'us-ny-031',
    'franklin': 'us-ny-033',
    'fulton': 'us-ny-035',
    'genesee': 'us-ny-037',
    'greene': 'us-ny-039',
    'hamilton': 'us-ny-041',
    'herkimer': 'us-ny-043',
    'jefferson': 'us-ny-045',
    'kings': 'us-ny-047',
    'lewis': 'us-ny-049',
    'livingston': 'us-ny-051',
    'madison': 'us-ny-053',
    'monroe': 'us-ny-055',
    'montgomery': 'us-ny-057',
    'nassau': 'us-ny-059',
    'new york': 'us-ny-061',
    'niagara': 'us-ny-063',
    'oneida': 'us-ny-065',
    'onondaga': 'us-ny-067',
    'ontario': 'us-ny-069',
    'orange': 'us-ny-071',
    'orleans': 'us-ny-073',
    'oswego': 'us-ny-075',
    'otsego': 'us-ny-077',
    'putnam': 'us-ny-079',
    'queens': 'us-ny-081',
    'rensselaer': 'us-ny-083',
    'richmond': 'us-ny-085',
    'rockland': 'us-ny-087',
    'saratoga': 'us-ny-091',
    'schenectady': 'us-ny-093',
    'schoharie': 'us-ny-095',
    'schuyler': 'us-ny-097',
    'seneca': 'us-ny-099',
    'st lawrence': 'us-ny-089',
    'steuben': 'us-ny-101',
    'suffolk': 'us-ny-103',
    'sullivan': 'us-ny-105',
    'tioga': 'us-ny-107',
    'tompkins': 'us-ny-109',
    'ulster': 'us-ny-111',
    'warren': 'us-ny-113',
    'washington': 'us-ny-115',
    'wayne': 'us-ny-117',
    'westchester': 'us-ny-119',
    'wyoming': 'us-ny-121',
    'yates': 'us-ny-123'
}

save_location = 'src/assets/'

In [3]:
df = pd.read_csv('Food_Service_Establishment__Last_Inspection.csv', sep=',')
df.head()

,FACILITY,ADDRESS,LAST INSPECTED,VIOLATIONS,TOTAL # CRITICAL VIOLATIONS,TOTAL #CRIT. NOT CORRECTED,TOTAL # NONCRITICAL VIOLATIONS,DESCRIPTION,LOCAL HEALTH DEPARTMENT,COUNTY,FACILITY ADDRESS,CITY,ZIP CODE,NYSDOH GAZETTEER (1980),MUNICIPALITY,OPERATION NAME,PERMIT EXPIRATION DATE,PERMITTED (D/B/A),PERMITTED CORP. NAME,PERM. OPERATOR LAST NAME,PERM. OPERATOR FIRST NAME,NYS HEALTH OPERATION ID,INSPECTION TYPE,INSPECTION COMMENTS,FOOD SERVICE FACILITY STATE,Location1
0,BORDERLINE TAVERN,"244-18 JERICHO TURNPIKE, BELLEROSE",11/22/2016,"Item 8A- Food not protected during storage, preparation, display, transportation and service, from potential sources of contamination (e.g., food uncovered, mislabeled, stored on floor, missing or inadequate sneeze guards, food containers double stacked); Item 8E- Accurate thermometers not available or used to evaluate refrigerated or heated storage temperatures; Item 15A- Floors, walls, ceilings, not smooth, properly constructed, in disrepair, dirty surfaces; Item 16- Miscellaneous, Economic Violation, Choking Poster, Training.;",0.0,0.0,5.0,Food Service Establishment - Food Service Establishment,Nassau County,NASSAU,244-18 JERICHO TURNPIKE,BELLEROSE,11001,294200,HEMPSTEAD,BORDERLINE TAVERN,10/31/2018,FRANKIE & VINNIE'S,VIN FRANK INC,NaN,NaN,578764,Re-Inspection,NaN,NY,"(40.725838, -73.718589)"
1,DOUGHBOYS PIZZERIA,"36 EAST STATE STREET, Mt. Morris",11/29/2016,"Item 8A- Food not protected during storage, preparation, display, transportation and service, from potential sources of contamination (e.g., food uncovered, mislabeled, stored on floor, missing or inadequate sneeze guards, food containers double stacked); Item 15A- Floors, walls, ceilings, not smooth, properly constructed, in disrepair, dirty surfaces; Item 15B- Lighting and ventilation inadequate, fixtures not shielded, dirty ventilation hoods, ductwork, filters, exhaust fans;",0.0,0.0,4.0,Food Service Establishment - Food Service Establishment,Livingston County,LIVINGSTON,36 EAST STATE STREET,Mt. Morris,14510,255800,MT. MORRIS,DOUGHBOYS PIZZERIA,08/01/2018,NaN,NaN,COLLARD,JON,585700,Inspection,"TPHC waiver left at the facility. If choosing to hot hold pizza with TCS toppings (i.e. cooked vegtables, meat other than pepperoni, chicken), the pizza must be held at 140F or higher or completed waiver must be approved. Call LCDOH to discuss waiver",NY,"(42.727516, -77.868842)"
2,SBARRO'S,"3620 PALISADES CENTER DRIVE, WEST NYACK",11/29/2016,"Item 8E- Accurate thermometers not available or used to evaluate refrigerated or heated storage temperatures; Item 12C- Plumbing and sinks not properly sized, installed, maintained; equipment and floors not properly drained; Item 15B- Lighting and ventilation inadequate, fixtures not shielded, dirty ventilation hoods, ductwork, filters, exhaust fans; Item 16- Miscellaneous, Economic Violation, Choking Poster, Training.;",0.0,0.0,4.0,Food Service Establishment - Food Service Establishment,Rockland County,ROCKLAND,3620 PALISADES CENTER DRIVE,WEST NYACK,10994,435000,CLARKSTOWN,SBARRO'S,11/30/2017,NaN,SBARRO LLC,KARAM,DAVID,820148,Inspection,NaN,NY,"(41.095665, -73.959621)"
3,PALISADES SUBWAY,"3563 PALISADES CENTER DRIVE, WEST NYACK",11/29/2016,"Item 15B- Lighting and ventilation inadequate, fixtures not shielded, dirty ventilation hoods, ductwork, filters, exhaust fans; Item 16- Miscellaneous, Economic Violation, Choking Poster, Training.;",0.0,0.0,2.0,Food Service Establishment - Food Service Establishment,Rockland County,ROCKLAND,3563 PALISADES CENTER DRIVE,WEST NYACK,10994,435000,CLARKSTOWN,PALISADES SUBWAY,11/30/2017,NaN,"SS FOOD SERVICE, INC.",AKHTER,SAEED,944671,Inspection,NaN,NY,"(41.095126, -73.958012)"
4,YEE OLDE HICKORY HOUSE,"3543 WATKINS ROAD, HORSEHEADS",11/29/2016,No violations found.,0.0,0.0,0.0,Food Service Establishment - Restaurant,Chemung County,CHEMUNG,3543 WATKINS ROAD,HORSEHEADS,14845,75300,CATLIN,YEE OLDE HICKORY HOUSE,08/15/2018,NaN,YEE OLDE HICKORY HOUSE INC,YEE,JOYCE,265397,Re-Inspection,"

In [4]:
df['LAST INSPECTED'] = pd.to_datetime(df['LAST INSPECTED'])
df['PERMIT EXPIRATION DATE'] = pd.to_datetime(df['PERMIT EXPIRATION DATE'], errors='coerce')
df = df.dropna(subset=['PERMIT EXPIRATION DATE','LAST INSPECTED'])

In [5]:
df['COUNTY'] = df['COUNTY'].str.lower()
df['CITY'] = df['CITY'].str.lower()
df['FACILITY'] = df['FACILITY'].str.lower()
df['ADDRESS'] = df['ADDRESS'].str.lower()
df['VIOLATIONS'] = df['VIOLATIONS'].str.lower()
df['INSPECTION COMMENTS'] = df['INSPECTION COMMENTS'].str.strip()
df['VIOLATIONS'] = df['VIOLATIONS'].str.strip()

In [6]:
df['REGION'] = df.apply(lambda row: regions[row['COUNTY']], axis=1)
df['GEOJSONID'] = df.apply(lambda row: geo_json_ids[row['COUNTY']], axis=1)

In [7]:
df['VIOLATIONS'] = df['VIOLATIONS'].str.replace('no violations found.', '')
df['SPLIT_VIOLATIONS'] = df['VIOLATIONS'].str.split(';')

In [8]:
for i in range(1,16):
    group_code_regex =  re.compile('item\s*({}+?[a-z])-'.format(i))
    df['VIOLATION_{}'.format(i)] = df.apply(lambda row: len(group_code_regex.findall(row['VIOLATIONS'])), axis=1)

In [9]:
df['COMMENT_LENGTH'] = df['INSPECTION COMMENTS'].str.len()
df['TOTAL # VIOLATIONS'] = df['SPLIT_VIOLATIONS'].str.len()
df['COMMENT_LENGTH_PER_VIOLATION'] = df.apply(lambda row: row['COMMENT_LENGTH'] / row['TOTAL # VIOLATIONS'], axis=1)

In [10]:
discriminators = ['REGION', 'COUNTY']

In [11]:
df['YEAR_WEEK'] = df['LAST INSPECTED'].dt.strftime('%Y/%U')

In [12]:
for discriminator in discriminators:
    result = df.groupby([df['YEAR_WEEK'], df[discriminator]]).mean().round()
#     min_year_month = result.reset_index()['YEAR_WEEK'].min()
#     max_year_month = result.reset_index()['YEAR_WEEK'].max()

    dates = [date.strftime('%Y/%U') for date in pd.date_range('2016/01', '2017/11', freq='1W')]
# #     # dates = [date.strftime('%Y/%m') for date in pd.date_range(min_year_month, max_year_month, freq='1M')]
    counties = df[discriminator].unique()
    iterables = [dates,counties]
    indexes = pd.MultiIndex.from_product(iterables, names=['YEAR_WEEK', discriminator])
    result = result.reindex(indexes, fill_value=np.nan)

    
    result = result.drop(['NYSDOH GAZETTEER (1980)', 'NYS HEALTH OPERATION ID'], axis=1)
    result = result.reset_index()
    
    if discriminator == 'COUNTY':
        result['REGION'] = result.apply(lambda row: regions[row['COUNTY']], axis=1)
        result['GEOJSONID'] = result.apply(lambda row: geo_json_ids[row['COUNTY']], axis=1)
    
    result = result.sort_values(by=['YEAR_WEEK', discriminator]).set_index(['YEAR_WEEK', discriminator])
    result.reset_index().to_json('{}data_per_{}_per_{}.json'.format(save_location, discriminator.lower(), 'week'), orient='records')

In [13]:
result = df.groupby([df['YEAR_WEEK']]).mean().round()

dates = [date.strftime('%Y/%U') for date in pd.date_range('2016/01', '2017/11', freq='1W')]
index = pd.Index(dates, name='YEAR_WEEK')
result = result.reindex(index)

result = result.drop(['NYSDOH GAZETTEER (1980)', 'NYS HEALTH OPERATION ID'], axis=1)

result = result.fillna(0, axis=1)
result = result.reset_index()

result.head()

result = result.sort_values(by=['YEAR_WEEK']).set_index(['YEAR_WEEK'])
result.head()
result.reset_index().to_json('{}data_per_week.json'.format(save_location), orient='records')

In [14]:
result = df.groupby([df['COUNTY']]).mean().round()

result = result.drop(['NYSDOH GAZETTEER (1980)', 'NYS HEALTH OPERATION ID'], axis=1)

result = result.reset_index()
result['GEOJSONID'] = result.apply(lambda row: geo_json_ids[row['COUNTY']], axis=1)

result = result.fillna(0, axis=1)
result = result.reset_index()

result.head()
result.reset_index().to_json('{}data_per_county.json'.format(save_location), orient='records')

In [15]:
violations = [[violation.strip() for violation in violations] for violations in df['SPLIT_VIOLATIONS']]
violations = list(itertools.chain.from_iterable(violations))

violations_df = pd.DataFrame({'short_code': [], 'group_code': [], 'long_code': [], 'count':[], 'critical': []})

violations_df['critical'] = violations_df['critical'].astype(bool)

pd.set_option('display.max_colwidth', 250)

counter = Counter()

counter.update(violations)

del counter['']

counter.most_common()

short_code_regex = re.compile('item\s+([a-z0-9]+)-')
group_code_regex =  re.compile('item\s*(\d+)\w*-')

for item in counter.most_common():
    violation = item[0]
    short_code = short_code_regex.search(violation)
    group_code = group_code_regex.search(violation)    
    critical = 'critical violation [red]' in violation
    
    if short_code:
        violation_df = pd.DataFrame({
            'short_code': [short_code.group(1)], 
            'group_code': [group_code.group(1)], 
            'long_code': [violation], 
            'count':[item[1]],
            'critical': [critical]
        })
        violations_df = violations_df.append(violation_df, ignore_index=True)

violations_df.head(250)

df['INSPECTION COMMENTS']

0                                                                                                                                                                                                                                                              NaN
1        TPHC waiver left at the facility. If choosing to hot hold pizza with TCS toppings (i.e. cooked vegtables, meat other than pepperoni, chicken), the pizza must be held at 140F or higher or completed waiver must be approved. Call LCDOH to discuss wa...
2                                                                                                                                                                                                                                                              NaN
3                                                                                                                                                                                                                              

In [16]:
df.to_csv('{}export.csv'.format(save_location), sep="=")
violations_df.to_csv('{}violations.csv'.format(save_location), sep="=")